<a href="https://colab.research.google.com/github/subinyounas/tensorflow/blob/main/Time_Series_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trying out various time series classfication models and comparing their performance. We have chosen 3 datasets from sktime.datasets.These are:
1) OSULeaf, ArrowHead and Gunpoint.
Various deep learning models are used to perform classification on these univariate time series.

In [ ]:
!pip install sktime


In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras-self-attention

In [ ]:
# loading neccessary imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional
import pandas as pd
from sktime.datasets import load_UCR_UEA_dataset
from sklearn.model_selection import train_test_split

c:\Users\subin\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# method to load different datasets
def load_data(dataset, split="train", shuffle=False, return_Type="numpy2D"):
    X, y = load_UCR_UEA_dataset(name=dataset, split=split, return_type=return_Type)
    y = keras.utils.to_categorical(y)  # convert to vector
    permutation_train = np.random.permutation(len(X))
    if shuffle:
        X = X[permutation_train]
        y = y[permutation_train]

    return X, y

# Process dataset - convert to tf.dataset, add shuffle , batch and prefetch
def process_dataset(X_train, y_train, X_test, y_test, BUFFER_SIZE=200, BATCH_SIZE=32):

    # convert to tensorflow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    train_dataset = train_dataset.shuffle(len(X_train))
    train_tf, valid_tf = keras.utils.split_dataset(
        train_dataset, left_size=0.75, shuffle=True
    )  # generate validation set
    # adding shuffle, batch, and prefetch
    train_tf_processed = train_tf.batch(BATCH_SIZE).prefetch(
        tf.data.experimental.AUTOTUNE
    )
    valid_tf_processed = valid_tf.batch(BATCH_SIZE)
    return train_tf_processed, valid_tf_processed, test_dataset

Created 4 deep learning models:-
1) CNN with 5 dense layers.
2) RNN
3) 1 Directional convolutional network
4) 1D CNN with GRU

In [ ]:
# Method to generate deep learning model
def generate_Model(nn, input_shape, no_of_classes):
    if nn == "fullyconnected":
        return keras.models.Sequential(
            [
                keras.layers.Input(shape=input_shape),
                keras.layers.Dense(128, activation="relu"),
                keras.layers.Dropout(0.2),
                keras.layers.Dense(128, activation="relu"),
                keras.layers.Dropout(0.2),
                keras.layers.Dense(64, activation="relu"),
                keras.layers.Dropout(0.2),
                keras.layers.Dense(32, activation="relu"),
                keras.layers.Dropout(0.2),
                keras.layers.Flatten(),
                keras.layers.Dense(no_of_classes, activation="softmax"),
            ],
            name=nn,
        )
    if nn == "rnn":
        return keras.models.Sequential(
            [
                keras.layers.SimpleRNN(128,input_shape = input_shape, activation="relu"),
                keras.layers.Dense(no_of_classes, activation="softmax"),
            ],
            name=nn,
        )
    if nn == "cnn":
        return keras.Sequential(
            [
                keras.layers.Input(shape=input_shape),
                keras.layers.Conv1D(
                    filters=64, kernel_size=3, activation="relu", padding="same"
                ),
                keras.layers.Conv1D(
                    filters=64, kernel_size=3, activation="relu", padding="same"
                ),
                keras.layers.MaxPooling1D(pool_size=1),
                keras.layers.Flatten(),
                keras.layers.Dense(128, activation="relu"),
                keras.layers.Dense(no_of_classes, activation="softmax"),
            ],
            name=nn,
        )
    if nn == "gru":
        model = keras.Sequential(
            [
                keras.layers.Input(shape=input_shape),
                keras.layers.Conv1D(
                    filters=64, kernel_size=3, activation="relu", padding="same"
                ),
                keras.layers.MaxPooling1D(pool_size=1),
                keras.layers.GRU(
                    units=128, activation='tanh', return_sequences=True
                ),
                keras.layers.Flatten(),
                keras.layers.Dense(units=32,activation='relu'),
                keras.layers.Dense(units=no_of_classes, activation="softmax"),
            ],
            name=nn,
        )
        return model

In [ ]:
datasets = ["GunPoint", "ArrowHead" ,"OSULeaf"]
models = [
    "gru",
    "fullyconnected",
    "rnn",
    "cnn",
]

In [ ]:
# callbacks
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.00001)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=10)

In [ ]:
for model_name in models:
    average_score = 0
    for dataset in datasets:
      X_train, y_train = load_data(dataset,'train',shuffle=True,return_Type="numpy3D")
      X_test , y_test = load_data(dataset,'test',return_Type="numpy3D")
      num_of_classes = y_train.shape[1] #for output layer
      input_shape = X_train.shape[1:] #for input layer
      train_data,valid_data,test_data = process_dataset(X_train,y_train,X_test,y_test)

      model = generate_Model(model_name,input_shape,num_of_classes)
      model.compile(
          optimizer=keras.optimizers.Adam(learning_rate=0.001),
          loss=keras.losses.CategoricalCrossentropy(),
          metrics=[
              keras.metrics.CategoricalAccuracy(),
          ],
      )
      model.fit(train_data, epochs=50, validation_data=valid_data, callbacks=[reduce_lr, early_stopping], verbose=0)
      result = model.evaluate(test_data.batch(32), return_dict=True)
      average_score += result['categorical_accuracy'] / len(datasets)
    print(f'Average accuracy for {model_name}: {average_score}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.6961 - loss: 0.6122 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.5998 - loss: 0.8532 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.4180 - loss: 1.4541
Average accuracy for gru: 0.5827285746733347
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.7261 - loss: 0.5315 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.6220 - loss: 0.8785
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.3705 - loss: 1.4673
Average accuracy for fullyconnected: 0.5642030835151672


c:\Users\subin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - categorical_accuracy: 0.7275 - loss: 0.5149  


c:\Users\subin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - categorical_accuracy: 0.6870 - loss: 0.7212  


c:\Users\subin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.4697 - loss: 1.4162
Average accuracy for rnn: 0.6429699659347534
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.7463 - loss: 0.5744
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.5629 - loss: 0.9603 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - categorical_accuracy: 0.4633 - loss: 1.5132
Average accuracy for cnn: 0.585606724023819


Comparison of CNN classifier from Sktime and two baseline deep learning models MLP and FCN.
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455) - Reference for baseline model

In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier

In [ ]:
reduce_lr_cnn = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3, min_lr=0.00001)
early_stopping_cnn = keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=10,mode="min")

In [ ]:
average_score = 0
for dataset in datasets:
    X_train, y_train = load_UCR_UEA_dataset(
        name=dataset, split="train", return_type="numpy2D"
    )
    X_test, y_test = load_UCR_UEA_dataset(
        name=dataset, split="test", return_type="numpy2D"
    )
    model = CNNClassifier(
        n_epochs=60,
        batch_size=32,
        callbacks=[reduce_lr_cnn, early_stopping_cnn],
    )
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"Categorical score for CNN on dataset '{dataset}' is:", score)
    average_score += score / len(datasets)
print("Average score for CNN classifier is:", average_score)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Categorical score for CNN on dataset 'GunPoint' is: 0.8
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Categorical score for CNN on dataset 'ArrowHead' is: 0.5085714285714286
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Categorical score for CNN on dataset 'OSULeaf' is: 0.5
Average score for CNN classifier is: 0.6028571428571429


In [ ]:
def get_model(model_name, input_shape, num_of_classes):
    if model_name == "mlp":
        return keras.Sequential(
            [
                keras.layers.Input(shape=input_shape, name="Input"),
                keras.layers.Dropout(rate=0.1),
                keras.layers.Dense(units=500, activation="relu"),
                keras.layers.Dropout(rate=0.2),
                keras.layers.Dense(units=500, activation="relu"),
                keras.layers.Dropout(rate=0.2),
                keras.layers.Dense(units=500, activation="relu"),
                keras.layers.Dropout(rate=0.3),
                keras.layers.Flatten(),
                keras.layers.Dense(num_of_classes, activation="softmax"),
            ],
            name=model_name,
        )
    elif model_name == "fcn":
        return keras.Sequential(
            [
                keras.layers.Input(shape=input_shape, name="Input"),
                keras.layers.Conv1D(
                    128, activation="relu", kernel_size=8, padding="same"
                ),
                keras.layers.BatchNormalization(),
                keras.layers.Conv1D(
                    256, activation="relu", kernel_size=5, padding="same"
                ),
                keras.layers.BatchNormalization(),
                keras.layers.Conv1D(
                    128, activation="relu", kernel_size=3, padding="same"
                ),
                keras.layers.BatchNormalization(),
                keras.layers.GlobalAveragePooling1D(),
                keras.layers.Flatten(),
                keras.layers.Dense(num_of_classes, activation="softmax", name="Output"),
            ],
            name=model_name,
        )

In [ ]:
model_list = ["mlp", "fcn"]

In [ ]:
for model_name in model_list:
    average_score = 0
    for dataset in datasets:
        X_train, y_train = load_data(dataset, "train", shuffle=True,return_Type="numpy3D")
        X_test, y_test = load_data(dataset, "test",return_Type="numpy3D")
        num_of_classes = y_train.shape[1]  # for output layer
        input_shape = X_train.shape[1:]  # for input layer
        train_data, valid_data, test_data = process_dataset(
            X_train, y_train, X_test, y_test
        )
        model = get_model(model_name, input_shape, num_of_classes)
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=[
                keras.metrics.CategoricalAccuracy(),
            ],
        )
        model.fit(
            train_data,
            epochs=60,
            validation_data=valid_data,
            callbacks=[reduce_lr, early_stopping],
            verbose=0,
        )
        result = model.evaluate(test_data.batch(32), return_dict=True)
        average_score += result["categorical_accuracy"] / len(datasets)
    print(f"Average accuracy for {model_name}: {average_score}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - categorical_accuracy: 0.7162 - loss: 1.0128
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.6506 - loss: 0.8929
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.4556 - loss: 1.5217 
Average accuracy for mlp: 0.6142804821332296
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - categorical_accuracy: 0.6487 - loss: 0.9573  
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.6712 - loss: 1.0224
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.4158 - loss: 1.6731 
Average accuracy for fcn: 0.521038959423701


Using a concatenated deep learning model by combining Bi directional lstm and 1D convulation network

In [ ]:
# using Bi direction LSTM and CNN on the data

In [ ]:
def get_bi_lstm_layer(input):
    return Sequential(
        [
            keras.layers.Bidirectional(keras.layers.LSTM(200, return_sequences=True)),
        ]
    )(input)

In [ ]:
def get_cnn_layer(input):
    conv_layers = Sequential(
        [
            keras.layers.Conv1D(200, kernel_size=8, padding="same", activation="relu"),
        ]
    )(input)
    return conv_layers

In [ ]:
def get_combined_model(input_shape, num_of_classes):
    input = keras.layers.Input(input_shape)
    cnn_layer = get_cnn_layer(input)

    lstm_layer = get_bi_lstm_layer(input)

    combined_layer = keras.layers.Concatenate()([cnn_layer, lstm_layer])

    outputs = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(combined_layer)
    outputs = keras.layers.GlobalMaxPooling1D()(outputs)
    outputs = keras.layers.Dense(num_of_classes, activation='softmax')(outputs)

    model = tf.keras.Model(inputs=input, outputs=outputs)
    return model

In [ ]:
average_score = 0
for dataset in datasets:
    X_train, y_train = load_data(dataset, "train", shuffle=True,return_Type="numpy3D")
    X_test, y_test = load_data(dataset, "test",return_Type="numpy3D")
    num_of_classes = y_train.shape[1]  # for output layer
    input_shape = X_train.shape[1:]  # for input layer
    train_data, valid_data, test_data = process_dataset(
        X_train, y_train, X_test, y_test
    )

    model = get_combined_model(input_shape, num_of_classes)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=[
            keras.metrics.CategoricalAccuracy(),
        ],
    )
    model.fit(
        train_data,
        epochs=60,
        validation_data=valid_data,
        callbacks=[reduce_lr, early_stopping],
        verbose=0,
    )
    result = model.evaluate(test_data.batch(32), return_dict=True)
    average_score += result["categorical_accuracy"] / len(datasets)
print(f"Average accuracy for combined model: {average_score}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.6983 - loss: 0.4489
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.7011 - loss: 0.7166
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.4901 - loss: 1.3759 
Average accuracy for combined model: 0.6371481021245321


Using transformers from sktime to transform the data before classifying with sktime CNN classifier

In [ ]:
#using sktime transformer + deeplearning (CNN Classifier)

In [ ]:
from sktime.transformations.series.impute import Imputer
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sklearn.pipeline import Pipeline

differencer = Imputer(method="drift")
box_cox = BoxCoxTransformer()

In [ ]:
def transform(X_train,X_test):
    pipe = Pipeline(steps=[("differencer", differencer), ("boxcox", box_cox)])

    # transform X-train
    X_train_transformed = pipe.fit_transform(X_train)
    X_test_transformed = pipe.transform(X_test)
    return X_train_transformed,X_test_transformed

In [ ]:
average_score = 0
for dataset in datasets:
    X_train, y_train = load_UCR_UEA_dataset(
        name=dataset, split="train", return_type="numpy2D"
    )
    X_test, y_test = load_UCR_UEA_dataset(
        name=dataset, split="test", return_type="numpy2D"
    )
    X_train_transformed, X_test_transformed = transform(X_train, X_test)
    model = CNNClassifier(
        n_epochs=60, batch_size=32, callbacks=[reduce_lr_cnn, early_stopping_cnn]
    )
    model.fit(X_train_transformed, y_train)
    score = model.score(X_test_transformed, y_test)
    print(f"Categorical score for CNN on dataset '{dataset}' is:", score)
    average_score += score / len(datasets)
print("Average score for CNN classifier is:", average_score)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Categorical score for CNN on dataset 'GunPoint' is: 0.5466666666666666
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Categorical score for CNN on dataset 'ArrowHead' is: 0.5942857142857143
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Categorical score for CNN on dataset 'OSULeaf' is: 0.371900826446281
Average score for CNN classifier is: 0.5042844024662206


Using SklearnClassifierPipeline to combine sktime transformers and an sklearn classifier. Combined the Rocket and Exponent transformer with RandomForest classfier to make a pipeline.
Random search to find the best model and parameters

In [ ]:
from sktime.classification.compose import SklearnClassifierPipeline
from sklearn.ensemble import RandomForestClassifier
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.series.exponent import ExponentTransformer
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
exponent = ExponentTransformer()
rocket = Rocket()
classifier = RandomForestClassifier()
pipeline = SklearnClassifierPipeline(
    classifier, transformers=[("exponent", exponent), ("rocket", rocket)]
)

In [ ]:
param_grid = {
    "classifier__max_depth": [3, 5, 7, 9],
    "rocket__normalise": [True, False],
    "exponent__power": [1, 2, 3, 4, 5],
}

In [ ]:
dataset = datasets[2]
X_train, y_train = load_UCR_UEA_dataset(
    name=dataset, split="train", return_type="numpy2D"
)
X_test , y_test = load_UCR_UEA_dataset(
    name=dataset, split="test", return_type="numpy2D"
)

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=3,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    error_score="raise"
)
random_search.fit(X_train, y_train)
print(f"Best score:{random_search.best_score_}") #best score form Random Search



Test score of the best model:0.7899894467058646


In [ ]:
print(f"Best params:{random_search.best_params_}") #best params form Random Search


Best params:{'rocket__normalise': False, 'exponent__power': 3, 'classifier__max_depth': 5}


In [ ]:
print(f"Best estimator:{random_search.best_estimator_}") #best estimator form Random Search
best_estimator = random_search.best_estimator_
best_estimator.score(X_test,y_test)

Best estimator:SklearnClassifierPipeline(classifier=RandomForestClassifier(max_depth=5),
                          transformers=[('exponent',
                                         ExponentTransformer(power=3)),
                                        ('rocket', Rocket(normalise=False))])


0.7396694214876033

In [ ]:
# getting the score of the default pipeline on the test set
pipe = SklearnClassifierPipeline(
    RandomForestClassifier(), transformers=[("exponent", ExponentTransformer()), ("rocket", Rocket())]
)
pipe.fit(X_train, y_train)
score = pipe.score(X_test, y_test)
print(f"Score on default pipeline:{score}")

Score on default pipeline:0.8057851239669421


Time series classification of Multivariate time series. Loaded the BasicMotions time series and performed classfication using Tapnet and Rocket from sktime and with MLP deep learning model. Compared the test scores.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sktime.classification.deep_learning.tapnet import TapNetClassifier
from sktime.classification.kernel_based import RocketClassifier

In [ ]:
label_encoder = LabelEncoder() #used to convert categories to int

In [ ]:
dataset = "BasicMotions"
X_train, y_train = load_UCR_UEA_dataset(
    name=dataset, split="train", return_type="numpy3D"
)
X_test, y_test = load_UCR_UEA_dataset(
    name=dataset, split="test", return_type="numpy3D"
)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.fit_transform(y_test)

In [ ]:
tapnet = TapNetClassifier()
tapnet.fit(X_train,y_train_encoded)
tapnet_score = tapnet.score(X_test, y_test_encoded)
print(f"Score of tapnet classifier:{tapnet_score}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step
Score of tapnet classifier:1.0


In [ ]:
rocket = RocketClassifier()
rocket.fit(X_train,y_train_encoded)
rocket_score = rocket.score(X_test, y_test_encoded)
print(f"Score of rocket classifier:{rocket_score}")

Score of rocket classifier:1.0


In [ ]:
y_train_one_hot = keras.utils.to_categorical(y_train_encoded)
y_test_one_hot = keras.utils.to_categorical(y_test_encoded)
train_data ,valid_data,test_data = process_dataset(X_train,y_train_one_hot,X_test,y_test_one_hot)

In [ ]:
# for performing classification with deep learning model
input_shape = X_train.shape[1:]
num_of_classes = len(np.unique(y_train_encoded))

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape, name="Input"),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(units=500, activation="relu"),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units=500, activation="relu"),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units=500, activation="relu"),
        keras.layers.Dropout(rate=0.3),
        keras.layers.Flatten(),
        keras.layers.Dense(num_of_classes, activation="softmax"),
    ],
    name="mlp",
)
model.compile(

    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[
        keras.metrics.CategoricalAccuracy(),
    ],
)



model.fit(
    train_data,
    epochs=50,
    validation_data=valid_data,
    callbacks=[reduce_lr, early_stopping],
    verbose=0,
)


result = model.evaluate(test_data.batch(32), return_dict=True)


score = result["categorical_accuracy"]


print(f"Test score of the deep learning Model:{score}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - categorical_accuracy: 0.5667 - loss: 1.0540
Test score of the deep learning Model:0.6000000238418579
